In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('jeopardy.csv')

df.head()

,Show Number,Air Date,Round,Category,Value,Question,Answer
0,4680,2004-12-31,Jeopardy!,HISTORY,$200,"For the last 8 years of his life, Galileo was ...",Copernicus
1,4680,2004-12-31,Jeopardy!,ESPN's TOP 10 ALL-TIME ATHLETES,$200,No. 2: 1912 Olympian; football star at Carlisl...,Jim Thorpe
2,4680,2004-12-31,Jeopardy!,EVERYBODY TALKS ABOUT IT...,$200,The city of Yuma in this state has a record av...,Arizona
3,4680,2004-12-31,Jeopardy!,THE COMPANY LINE,$200,"In 1963, live on ""The Art Linkletter Show"", th...",McDonald's
4,4680,2004-12-31,Jeopardy!,EPITAPHS & TRIBUTES,$200,"Signer of the Dec. of Indep., framer of the Co...",John Adams


## Clean all columns names and blank spaces

In [2]:
df.columns = df.columns.str.lower()
df.columns = df.columns.str.replace(' ','')

In [3]:
df.columns

Index(['shownumber', 'airdate', 'round', 'category', 'value', 'question',
       'answer'],
      dtype='object')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19999 entries, 0 to 19998
Data columns (total 7 columns):
shownumber    19999 non-null int64
airdate       19999 non-null object
round         19999 non-null object
category      19999 non-null object
value         19999 non-null object
question      19999 non-null object
answer        19999 non-null object
dtypes: int64(1), object(6)
memory usage: 1.1+ MB


write a function that normalizes questions and answers:

1. take in a string
2. covert string to lowercase
3. remove all punctuation
4. return the string

## Remove all punctuation and lowercase each item in a column

In [5]:
import string

def remove_punctuation(text):
    result = ""
    punctuations = np.array([c for c in string.punctuation])
    for char in text:
        if char not in punctuations:
            result += char.lower()
    return result

## Clean mulitple columns at once

df[['question','answer']] = df[['question','answer']].apply(remove_punctuation)

In [6]:
df.question = df.question.apply(remove_punctuation)

In [7]:
df.answer = df.answer.apply(remove_punctuation)

In [8]:
df.head()

,shownumber,airdate,round,category,value,question,answer
0,4680,2004-12-31,Jeopardy!,HISTORY,$200,for the last 8 years of his life galileo was u...,copernicus
1,4680,2004-12-31,Jeopardy!,ESPN's TOP 10 ALL-TIME ATHLETES,$200,no 2 1912 olympian football star at carlisle i...,jim thorpe
2,4680,2004-12-31,Jeopardy!,EVERYBODY TALKS ABOUT IT...,$200,the city of yuma in this state has a record av...,arizona
3,4680,2004-12-31,Jeopardy!,THE COMPANY LINE,$200,in 1963 live on the art linkletter show this c...,mcdonalds
4,4680,2004-12-31,Jeopardy!,EPITAPHS & TRIBUTES,$200,signer of the dec of indep framer of the const...,john adams


## Normalizing Value columns

write a function that takes in a string and:

1. removes any punctuation in the string
2. converts the string to an integer
3. assigns 0 instead if conversion has an error
4. return the integer

In [9]:
def normalize_value(column):
    column = column.translate(str.maketrans('','', string.punctuation))
    try:
        result = int(column)
    except:
        result = 0
    return result
    
        

In [10]:
df['value'] = df['value'].apply(normalize_value)

## Convert to datetime column

In [11]:
df.airdate = pd.to_datetime(df.airdate)

Create a function that:
1. splits column around spaces and assigns the variable to split answer
2. split the question around spaces and assigns the variable to split question
3. create a variable called match count and set it equal to 0
4. if 'the' in split answer, remove it
5. divide the match count by the length of the split answer

In [12]:
def count_matches(row):
    split_answer = row['answer'].split()
    split_question = row['question'].split()
    if 'the' in split_answer:
        split_answer.remove('the')
    if len(split_answer) == 0:
        return 0
    match_count = 0
    for item in split_answer:
        if item in split_question:
            match_count += 1
    return match_count / len(split_answer)

In [13]:
df['answers_in_questions'] = df.apply(count_matches, axis=1)

In [14]:
df.answers_in_questions.mean()

0.058861482035140716

this means that only 6% of answers are in the questions so we will probably have to study

## Find how often new items are repeats of old items

1. Sort df in order of date
2. new set called terms used
3. loop and assign to split question, split answer
4. remove words less than 6 characters 
5. match count = 0
6. loop through each word in split_question, if occured in terms used, add 1 to match
7. add each word used in question to set
8. if length of question is greater than 0, divide match_count by len of split question
9. append match_count to question_overlap


## For loop in one line of code
## Itterows on an entire dataset

In [32]:
question_overlap = []
terms_used = set()

df = df.sort_values('airdate', ascending=True)

for i, row in df.iterrows():
    split_question = row['question'].split(' ')
    split_question = [q for q in split_question if len(q) > 5]
    match_count = 0
    for word in split_question:
        if word in terms_used:
            match_count += 1
    for word in split_question:
        terms_used.add(word)
    if len(split_question) > 0:
            match_count /= len(split_question)
    question_overlap.append(match_count)
    
df['question_overlap'] = question_overlap
df['question_overlap'].mean()

0.687023624575748

this means theres a 70% overlap in single terms. Worth looking more into recycling of questins

# IMPORTANT : If you make a function and pass through a row as an input, you must apply the axis=1 

In [39]:
def value_maker(row):
    if row['value'] > 800:
        value = 1
    else:
        value = 0
    return value

In [40]:
df['high_value'] = df.apply(value_maker, axis=1)

In [41]:
df.head()

,shownumber,airdate,round,category,value,question,answer,answers_in_questions,question_overlap,high_value
19304,10,1984-09-21,Double Jeopardy!,LITERATURE,200,depending on the book he could be a jones a sa...,tom,0.0,0.0,0
19318,10,1984-09-21,Double Jeopardy!,HOMONYMS,1000,didnt see the fog,missed mist,0.0,0.0,1
19317,10,1984-09-21,Double Jeopardy!,LITERATURE,800,irvings birdlike hero chased by the headless h...,ichabod crane,0.0,0.0,0
19311,10,1984-09-21,Double Jeopardy!,TV TRIVIA,400,he went from the ponderosa to the prairie,michael landon,0.0,0.0,0
19297,10,1984-09-21,Jeopardy!,"""JACKS"" OF ALL TRADES",500,2nd generation pop singer hes sung on every lo...,jack jones,0.0,0.0,0


Create a function that:
1. takes in a word and assigns 0 to low count and 0 to high count
2. loops through each row in jeopardy using iterrows
3. splits question on the space
4. if the word is in split question, if the high count value is 1, add 1 to high count, else 1 to low count
5. return high and low count

In [43]:
def high_or_low(word):
    low_count = 0
    high_count = 0
    for i, row in df.iterrows():
        split_question = row['question'].split(' ')
        if word in split_question:
            if row['high_value'] == 1:
                high_count += 1
            else:
                low_count += 1
    return low_count, high_count

randomly pick 10 elements of terms used and append them to a list called comparison terms

In [44]:
import random

comparision_terms = random.sample(terms_used, 10)

In [47]:
observed_expected = []
for item in comparision_terms:
    observed_expected.append(high_or_low(item))
    